In [2]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

In [3]:
# Load Data
dbt = pd.read_csv('../assets/diabetes.csv')

dbt.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
# Cek nama kolom
dbt.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [5]:
# Cek kolom null
dbt.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [6]:
# Pada kasus ini, agak tidak masuk akal jika beberapa parameter bernilai 0
# sebagai contoh adalah nilai 'Glucose', 'BloodPlessure' ataupun 'Insulin'.
# Sekecil apapun nilainya, setiap manusia yang hidup pasti miliki nilai-nilai tersebut

# Kita akan manipulasi nilai yang 0 dengan melakukan 'imputasi' atau mengganti nilainya dengan nilai sintetis
# Pada kasus ini, kita akan menggunakan nilai mean

# Cek kolom neng nilai 0
feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
for column in feature_columns:
    print("============================================")
    print(f"{column} ==> Missing zeros : {len(dbt.loc[dbt[column] == 0])}")

Pregnancies ==> Missing zeros : 111
Glucose ==> Missing zeros : 5
BloodPressure ==> Missing zeros : 35
SkinThickness ==> Missing zeros : 227
Insulin ==> Missing zeros : 374
BMI ==> Missing zeros : 11
DiabetesPedigreeFunction ==> Missing zeros : 0
Age ==> Missing zeros : 0


In [7]:
# Impute nilai 0 dengan mean
from sklearn.impute import SimpleImputer

fill_values = SimpleImputer(missing_values=0, strategy="mean", copy=False)

dbt[feature_columns] = fill_values.fit_transform(dbt[feature_columns])

In [8]:
X = dbt[feature_columns]
y = dbt.Outcome

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
# Karena asumsi Gaussian NB adalah data terdistribusi secara normal,
# maka kita perlu melakukan standarisasi

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

# Standarisasi pada fitur di X_train dan X_test
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

### Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression

# Membuat model Logistic Regression
logreg_model = LogisticRegression(max_iter=1000, random_state=42)

# Fit the model on the training data
logreg_model.fit(X_train, y_train)

from sklearn.metrics import confusion_matrix, accuracy_score

# Memprediksi label untuk data uji
y_pred = logreg_model.predict(X_test)

# Menghitung dan mencetak akurasi
test_accuracy = accuracy_score(y_test, y_pred)
print(f'\nLogistic Regression Accuracy: {test_accuracy}')
print(f"Logistic Regression Accuracy: {test_accuracy*100:.2f}%")


Logistic Regression Accuracy: 0.7359307359307359
Logistic Regression Accuracy: 73.59%


### SVM Polynomial

In [17]:
# Model SVM polynomial tanpa tunnning hyperparameter
svm_pol = SVC(kernel='poly')

# Fit ke model
svm_pol.fit(X_train_std, y_train)

# Prediksi
y_pred_svm_pol = svm_pol.predict(X_test_std)

# Evaluasi akurasi testing data
acc_svm_pol = accuracy_score(y_test, y_pred_svm_pol)

# Print hasil evaluasi
print(f"SVM Polynomial Accuracy: {acc_svm_pol}")
print(f"SVM Polynomial Accuracy: {acc_svm_pol*100:.2f}%")

SVM Polynomial Accuracy: 0.696969696969697
SVM Polynomial Accuracy: 69.70%


### Decision Tree

In [19]:
# Secara default, DecisionTreeClassifier dari scikit-learn akan menggunakan nilai "Gini" untuk kriteria
# Terdapat beberapa "hyperparamater" yang dapat digunakan. Silahka baca dokumentasi
# Pada kasus ini kita akan menggunakan parameter default
dt = DecisionTreeClassifier()

# Sesuaikan dt ke set training
dt.fit(X_train, y_train)

# Memprediksi label set test
y_pred_dt = dt.predict(X_test)

#  menghitung set accuracy
acc_dt = accuracy_score(y_test, y_pred_dt)
print(f"Decission Tree Accuracy: {acc_dt}")
print(f"Decission Tree Accuracy: {acc_dt*100:.2f}%")

Decission Tree Accuracy: 0.696969696969697
Decission Tree Accuracy: 69.70%


In [22]:
# Print classification reports
print("Logistic Regression Classification Report:\n", classification_report(y_test, y_pred))
print("SVM Polymol Classification Report:\n", classification_report(y_test, y_pred_svm_pol))
print("Decision Tree Classification Report:\n", classification_report(y_test, y_pred_dt))

Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.81      0.80       151
           1       0.63      0.59      0.61        80

    accuracy                           0.74       231
   macro avg       0.71      0.70      0.70       231
weighted avg       0.73      0.74      0.73       231

SVM Polymol Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.88      0.79       151
           1       0.61      0.35      0.44        80

    accuracy                           0.70       231
   macro avg       0.66      0.62      0.62       231
weighted avg       0.68      0.70      0.67       231

Decision Tree Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.75      0.77       151
           1       0.56      0.59      0.57        80

    accuracy                           0.70       231
   macro a